In [1]:
import wandb
import pandas as pd

ENTITY = "jacoba-california-state-university-east-bay"
PROJECT = "humaid_ssl"
SWEEP_FILE = "../secrets/sweep_ids.txt"  # one sweep ID per line

api = wandb.Api()

# --- Read sweep IDs ---
with open(SWEEP_FILE) as f:
    SWEEP_IDS = [line.strip() for line in f if line.strip()]

records = []

print(f"Fetching runs from {len(SWEEP_IDS)} sweeps...")

for sweep_id in SWEEP_IDS:
    sweep = api.sweep(f"{ENTITY}/{PROJECT}/{sweep_id}")
    for run in sweep.runs:
        s = run.summary

        macro_f1_final = s.get("macro-F1")
        macro_f1_std   = s.get("macro-F1_std")

        # === Extract subrun-level F1s ===
        hist = run.history(samples=10000)
        if {"sub_macro-F1", "set_num", "seed"}.issubset(hist.columns):
            subruns = (
                hist[["set_num", "seed", "sub_macro-F1", "sub_acc"]]
                .dropna(subset=["sub_macro-F1"])
            )
            best_sub_f1 = subruns["sub_macro-F1"].max() if len(subruns) > 0 else None
        else:
            best_sub_f1 = None

        records.append({
            "sweep_id": sweep_id,
            "run_id": run.id,
            "run_name": run.name,
            "macro_F1_final": macro_f1_final,
            "macro_F1_std": macro_f1_std,
            "best_subrun_F1": best_sub_f1
        })

# --- Combine into one DataFrame ---
df = pd.DataFrame(records)

# --- Select best run per sweep ---
best_per_sweep = (
    df.sort_values(["sweep_id", "best_subrun_F1"], ascending=[True, False])
      .groupby("sweep_id", as_index=False)
      .first()
)

print("\n=== Best runs per sweep ===")
print(best_per_sweep[["run_name", "best_subrun_F1", "macro_F1_final"]])

print(f"\nTotal sweeps processed: {len(SWEEP_IDS)}")
print(f"Total runs processed: {len(df)}")

def parse_event_lbcl(name):
    name = str(name)
    try:
        # e.g. "HumAID-hurricane_florence_2018-lb10-1763573758"
        parts = name.split("-")
        # get lb token (e.g. lb10)
        lb = next((p for p in parts if p.startswith("lb")), "lb?")
        # get event (everything between first '-' and lb token)
        event = "-".join(parts[1:parts.index(lb)]) if lb in parts else "-".join(parts[1:-1])
    except Exception:
        event, lb = name, "lb?"
    return event, lb

df = best_per_sweep.copy()
df[["event", "lbcl"]] = df["run_name"].apply(lambda n: pd.Series(parse_event_lbcl(n)))

# --- Pivot: rows=event, columns=lbcl, values=best_subrun_F1 ---
pivot = (
    df.pivot_table(
        index="event",
        columns="lbcl",
        values="best_subrun_F1",
        aggfunc="max"
    )
    .sort_index()
)

print("\n=== sub_Macro-F1 Pivot (event × lbcl) ===")
print(pivot["lb50"])

# --- Pivot: rows=event, columns=lbcl, values=macro_F1_final ---
pivot = (
    df.pivot_table(
        index="event",
        columns="lbcl",
        values="macro_F1_final",
        aggfunc="max"
    )
    .sort_index()
)

print("\n=== final_Macro-F1 Pivot (event × lbcl) ===")
print(pivot["lb50"])

Fetching runs from 40 sweeps...

=== Best runs per sweep ===
                                            run_name  best_subrun_F1  \
0   HumAID-california_wildfires_2018-lb50-1763639566        0.696758   
1    HumAID-california_wildfires_2018-lb5-1763479249        0.482166   
2       HumAID-canada_wildfires_2016-lb10-1763463693        0.601639   
3        HumAID-canada_wildfires_2016-lb5-1763461031        0.577566   
4           HumAID-cyclone_idai_2019-lb25-1763554077        0.654854   
5       HumAID-canada_wildfires_2016-lb25-1763471915        0.670006   
6       HumAID-hurricane_harvey_2017-lb50-1763678301        0.679200   
7       HumAID-canada_wildfires_2016-lb50-1763610169        0.654685   
8           HumAID-cyclone_idai_2019-lb50-1763613132        0.681375   
9   HumAID-california_wildfires_2018-lb25-1763544709        0.659797   
10      HumAID-hurricane_dorian_2019-lb50-1763647485        0.639893   
11    HumAID-hurricane_florence_2018-lb25-1763579878        0.668510   
12 